In [3]:
import json
import zhconv
dataName = "Mafengwo"

def combineInfo(dict, attrList):
    rnt = []
    # 把感兴趣的属性提取出来
    for attr in attrList:
        rnt = rnt + dict.get(attr, [["", 0]])
    # 按照位置排序
    rnt = sorted(rnt, key=lambda x:x[1])
    # 提取文本
    rnt = [x[0] for x in rnt]
    # 合并文本
    return "".join(rnt)

In [4]:
with open("../user_data/%s.json"%dataName, "r", encoding="utf-8") as f:
    dataRough  = json.load(f)
with open("../user_data/%s_addr_result.json"%dataName, "r", encoding="utf-8") as f:
    dataDetail = json.load(f)

result = []
for i in range(len(dataRough)):
    d1 = dataRough[i]; d2 = dataDetail[i]
    tmp = {"id": dataName[0] + str(i)}

    # 粗粒度数据提取
    if d1["行政区"] is None or d1["行政区"] == "" or d1["行政区"] == "nan": tmp["行政区"] = "U"
    else: tmp["行政区"] = zhconv.convert(d1["行政区"], "zh-hans")
    if d1["Location"] == "nan":
        tmp["Location"] = [None, None]
    else:
        lst = d1["Location"].split(",")
        tmp["Location"] = [float(lst[0]), float(lst[1])]
    tmp["Name"] = d1["Name"]
    tmp["Address"] = d1["Address"]

    # 细粒度数据提取
    # 1. 处理大区域
    tmp["District"] = combineInfo(d2, ["district", "town", "community"])
    # 2. 处理道路信息
    tmp["RoadInfo"] = combineInfo(d2, ["road", "roadno"])
    # 3. 处理兴趣点信息
    tmp["Poi"] = combineInfo(d2, ["poi", "subpoi"])
    # 4. 处理楼栋、房号信息
    tmp["RoomInfo"] = combineInfo(d2, ["houseno", "cellno", "floorno", "roomno"])

    result.append(tmp)

with open("../user_data/%s_addr.json"%dataName, "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=2)
